In [2]:
from PIL import Image, ImageDraw
img = Image.open('images/IMG_0915.jpg')

In [11]:
center_x = int(img.width / 2)
center_y = int(img.height / 2)
upper_edge_y = None
for i in range(0, img.height):
    average_pixel_rgb = sum(img.getpixel((center_x, i))) / 3
    # print (average_pixel_rgb)
    if not (155 < average_pixel_rgb < 170):
        # We have likely hit the upper edge of the real image
        # draw = ImageDraw.Draw(img)
        # draw.line((0, i, img.width, i), fill=128)
        # img.show()
        upper_edge_y = i
        break

lower_edge_y = None
for i in range(img.height - 1, 0, -1):
    average_pixel_rgb = sum(img.getpixel((center_x, i))) / 3
    # print (average_pixel_rgb)
    if not (155 < average_pixel_rgb < 170):
        # We have likely hit the lower edge of the real image
        # draw = ImageDraw.Draw(img)
        # draw.line((0, i, img.width, i), fill=128)
        # img.show()
        lower_edge_y = i
        break
        
right_edge_x = None
for i in range(img.width - 1, 0, -1):
    average_pixel_rgb = sum(img.getpixel((i, center_y))) / 3
    # print (average_pixel_rgb)
    if not (155 < average_pixel_rgb < 170):
        # We have likely hit the left edge of the real image
        # draw = ImageDraw.Draw(img)
        # draw.line((i, 0, i, img.height), fill=128)
        # img.show()
        right_edge_x = i
        break
        
# TODO: Find where the sciency colorgram-thing ends and left gray area begins, from there do regular detection